In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm.notebook import tqdm

import cv2, os, pickle


import joblib
from multiprocessing import cpu_count

In [2]:
DEBUG = False
IMG_HEIGHT = 300
IMG_WIDTH = 480
VAL_SIZE = int(100) if DEBUG else int(100e3) # 100K validation molecules
CHUNK_SIZE = 40000 # to get ~100MB TFRecords

MAX_INCHI_LEN = 200 # maximum InChI length to prevent to much padding

In [3]:
if DEBUG:
    train = pd.read_csv('/kaggle/input/bms-molecular-translation/train_labels.csv', dtype={ 'image_id': 'string', 'InChI': 'string' }).head(int(1e3))
else:
    train = pd.read_csv('/kaggle/input/bms-molecular-translation/train_labels.csv', dtype={ 'image_id': 'string', 'InChI': 'string' })

# Drop all InChI longer than MAX_INCHI_LEN - 2,  <start>InChI <end>, remove 'InChI=1S/' at start
train['InChI_len'] = train['InChI'].apply(len).astype(np.uint16)
train = train.loc[train['InChI_len'] <= MAX_INCHI_LEN - 2 + 9].reset_index(drop=True)
train.head(3)

,image_id,InChI,InChI_len
0,000011a64c74,InChI=1S/C13H20OS/c1-9(2)8-15-13-6-5-10(3)7-12...,81
1,000019cc0cd2,InChI=1S/C21H30O4/c1-12(22)25-14-6-8-20(2)13(1...,155
2,0000252b6d2b,InChI=1S/C24H23N5O4/c1-14-13-15(7-8-17(14)28-1...,158


In [4]:
if DEBUG:
    test = pd.read_csv('/kaggle/input/bms-molecular-translation/sample_submission.csv', usecols=['image_id'], dtype={ 'image_id': 'string' }).head(int(1e3))
else:
    test = pd.read_csv('/kaggle/input/bms-molecular-translation/sample_submission.csv', usecols=['image_id'], dtype={ 'image_id': 'string' })
test.head(3)

,image_id
0,00000d2a601c
1,00001f7fc849
2,000037687605


In [5]:
def get_vocabulary():
    tokens = ['<start>', '<end>', '<pad>']
    vocabulary = set()
    for s in tqdm(train['InChI'].values):
        vocabulary.update(s)
    return tokens + list(vocabulary)

vocabulary = get_vocabulary()

  0%|          | 0/2371184 [00:00<?, ?it/s]

In [6]:
# Save vocabulary mappings
# character -> integer
vocabulary_to_int = dict(zip(vocabulary, np.arange(len(vocabulary), dtype=np.int8)))
with open('vocabulary_to_int.pkl', 'wb') as handle:
    pickle.dump(vocabulary_to_int, handle)

#  integer -> character
int_to_vocabulary = dict(zip(np.arange(len(vocabulary), dtype=np.int8), vocabulary))
with open('int_to_vocabulary.pkl', 'wb') as handle:
    pickle.dump(int_to_vocabulary, handle)

In [7]:
train['InChIClean'] = train['InChI'].apply(lambda InChI: '/'.join(InChI.split('=')[1].split('/')[1:]))
train.head()

,image_id,InChI,InChI_len,InChIClean
0,000011a64c74,InChI=1S/C13H20OS/c1-9(2)8-15-13-6-5-10(3)7-12...,81,C13H20OS/c1-9(2)8-15-13-6-5-10(3)7-12(13)11(4)...
1,000019cc0cd2,InChI=1S/C21H30O4/c1-12(22)25-14-6-8-20(2)13(1...,155,C21H30O4/c1-12(22)25-14-6-8-20(2)13(10-14)11-1...
2,0000252b6d2b,InChI=1S/C24H23N5O4/c1-14-13-15(7-8-17(14)28-1...,158,C24H23N5O4/c1-14-13-15(7-8-17(14)28-12-10-20(2...
3,000026b49b7e,InChI=1S/C17H24N2O4S/c1-12(20)18-13(14-7-6-10-...,147,C17H24N2O4S/c1-12(20)18-13(14-7-6-10-24-14)11-...
4,000026fc6c36,InChI=1S/C10H19N3O2S/c1-15-10(14)12-8-4-6-13(7...,96,C10H19N3O2S/c1-15-10(14)12-8-4-6-13(7-8)5-2-3-...


In [8]:
# convert the InChI strings to integer lists
# start/end/pad tokens are used
def inchi_str2int(InChI):
    res = []
    res.append(vocabulary_to_int.get('<start>'))
    for c in InChI:
        res.append(vocabulary_to_int.get(c))
    
    res.append(vocabulary_to_int.get('<end>'))
    while len(res) < MAX_INCHI_LEN: 
        res.append(vocabulary_to_int.get('<pad>'))
        
    return np.array(res, dtype=np.uint8)

tqdm.pandas() # progress_apply
train['InChI_int'] = train['InChIClean'].progress_apply(inchi_str2int)
train.head()

  0%|          | 0/2371184 [00:00<?, ?it/s]

,image_id,InChI,InChI_len,InChIClean,InChI_int
0,000011a64c74,InChI=1S/C13H20OS/c1-9(2)8-15-13-6-5-10(3)7-12...,81,C13H20OS/c1-9(2)8-15-13-6-5-10(3)7-12(13)11(4)...,"[0, 24, 20, 11, 32, 18, 35, 38, 9, 22, 31, 20,..."
1,000019cc0cd2,InChI=1S/C21H30O4/c1-12(22)25-14-6-8-20(2)13(1...,155,C21H30O4/c1-12(22)25-14-6-8-20(2)13(10-14)11-1...,"[0, 24, 18, 20, 32, 11, 35, 38, 6, 22, 31, 20,..."
2,0000252b6d2b,InChI=1S/C24H23N5O4/c1-14-13-15(7-8-17(14)28-1...,158,C24H23N5O4/c1-14-13-15(7-8-17(14)28-12-10-20(2...,"[0, 24, 18, 6, 32, 18, 11, 36, 15, 38, 6, 22, ..."
3,000026b49b7e,InChI=1S/C17H24N2O4S/c1-12(20)18-13(14-7-6-10-...,147,C17H24N2O4S/c1-12(20)18-13(14-7-6-10-24-14)11-...,"[0, 24, 20, 10, 32, 18, 6, 36, 18, 38, 6, 9, 2..."
4,000026fc6c36,InChI=1S/C10H19N3O2S/c1-15-10(14)12-8-4-6-13(7...,96,C10H19N3O2S/c1-15-10(14)12-8-4-6-13(7-8)5-2-3-...,"[0, 24, 20, 35, 32, 20, 14, 36, 11, 38, 18, 9,..."


In [9]:
val = train.iloc[-VAL_SIZE:].reset_index(drop=True)
train = train.iloc[:-VAL_SIZE].reset_index(drop=True)
N_IMGS = len(train)

In [10]:
# plt.figure(figsize=(14, 14))
# img= cv2.imread('../input/bms-molecular-translation/train/d/9/e/d9e032a94a24.png', 0)
# plt.imshow(img)
# plt.show()

In [11]:
def process_img(image_id, folder='train', debug=False):
    # read image and invert colors to get black background and white molecule
    file_path =  f'/kaggle/input/bms-molecular-translation/{folder}/{image_id[0]}/{image_id[1]}/{image_id[2]}/{image_id}.png'
    img0 = 255 - cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    
    # rotate counter clockwise to get horizontal images
    h, w = img0.shape
    if h > w:
        img0 = np.rot90(img0)
        
    img = cv2.resize(img0,(IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_NEAREST)
    if debug:
        fig, ax = plt.subplots(1, 2, figsize=(20,10))
        ax[0].imshow(img0)
        ax[0].set_title('Original image', size=16)
        ax[1].imshow(img)
        ax[1].set_title('Fully processed image', size=16)
    
    # normalize to range 0-255 and encode as png
    img = (img / img.max() * 255).astype(np.uint8)
    img = cv2.imencode('.png', img)[1].tobytes()

    return img

In [12]:
def split_in_chunks(data):
    return [data[i:i + CHUNK_SIZE] for i in range(0, len(data), CHUNK_SIZE)]

train_data_chunks = {
    'train': {
        'image_id': split_in_chunks(train['image_id'].values),
        'InChI': split_in_chunks(train['InChI_int'].values),
    },
    'val': {
        'image_id': split_in_chunks(val['image_id'].values),
        'InChI': split_in_chunks(val['InChI_int'].values),
    }
}

test_data_chunks = {
    'test': {
        'image_id': split_in_chunks(test['image_id'].values),
    }
}

In [13]:
def make_tfrecords(data_chunks, folder='train'):
    # Try to make output folder
    try:
        os.makedirs(f'./train')
        os.makedirs(f'./val')
        os.makedirs(f'./test')
    except:
        print(f'folders already created')

    for k, v in data_chunks.items():
        for chunk_idx, image_id_chunk in tqdm(enumerate(v['image_id']), total=len(v['image_id'])):
            # process images in parallel
            jobs = [joblib.delayed(process_img)(fp, folder) for fp in image_id_chunk]
            bs = 10
            processed_images_chunk = joblib.Parallel(
                n_jobs=cpu_count(),
                verbose=0,
                require='sharedmem',
                batch_size=bs,
                backend='threading',
            )(jobs)

            # Create the TFRecords from the processed images
            with tf.io.TFRecordWriter(f'./{k}/batch_{chunk_idx}.tfrecords') as file_writer:
                if 'InChI' in v.keys(): # TRAIN/VAL, InChI included
                    for image, InChI in zip(processed_images_chunk, v['InChI'][chunk_idx]):
                        record_bytes = tf.train.Example(features=tf.train.Features(feature={
                            'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),
                            'InChI': tf.train.Feature(int64_list=tf.train.Int64List(value=InChI)),
                        })).SerializeToString()
                        file_writer.write(record_bytes)
                else: # TEST, image_id included for submission file
                    for image, image_id in zip(processed_images_chunk, image_id_chunk):
                        record_bytes = tf.train.Example(features=tf.train.Features(feature={
                            'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),
                            'image_id': tf.train.Feature(bytes_list=tf.train.BytesList(value=[str.encode(image_id)])),
                        })).SerializeToString()
                        file_writer.write(record_bytes)

make_tfrecords(train_data_chunks)
make_tfrecords(test_data_chunks, 'test')

  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

folders already created


  0%|          | 0/41 [00:00<?, ?it/s]

In [14]:
# convert in int encoded InChI to string
def inchi_int2char(InChI):
    res = []
    for i in InChI:
        c = int_to_vocabulary.get(i)
        if c not in ['<start>', '<end>', '<pad>']:
            res.append(c)
    return ''.join(res)

In [15]:
# Check train TFRecords
def decode_tfrecord(record_bytes):
    fea_dict= {
        'image': tf.io.FixedLenFeature([], tf.string),
        'InChI': tf.io.FixedLenFeature([MAX_INCHI_LEN], tf.int64),}
    
    features = tf.io.parse_single_example(record_bytes, fea_dict)

    image = tf.io.decode_jpeg(features['image'])    
    image = tf.reshape(image, [IMG_HEIGHT, IMG_WIDTH, 1])
    image = tf.cast(image, tf.float32)  / 255.0
    
    InChI = features['InChI']
    InChI = tf.reshape(InChI, [MAX_INCHI_LEN])
    
    return image, InChI


# Check test TFRecords
def decode_test_tfrecord(record_bytes):
    features = tf.io.parse_single_example(record_bytes, {
        'image': tf.io.FixedLenFeature([], tf.string),
        'image_id': tf.io.FixedLenFeature([], tf.string),
    })

    image = tf.io.decode_jpeg(features['image'])
    image = tf.reshape(image, [IMG_HEIGHT, IMG_WIDTH, 1])
    image = tf.cast(image, tf.float32)  / 255.0
    
    image_id = features['image_id']
    
    return image, image_id
